In [1]:
from mastodon import Mastodon, StreamListener
from kafka import KafkaProducer, KafkaAdminClient
from kafka.admin import NewTopic
import json
import time
import random
from datetime import datetime

# Connexion à l'API Mastodon
mastodon = Mastodon(
    access_token='',
    api_base_url='https://mastodon.social'  # Remplacez par l'URL correcte de votre instance
)

# Connexion à Kafka Admin pour la création de topics
admin_client = KafkaAdminClient(bootstrap_servers='host.docker.internal:9092', client_id='my_client')

# Définir le nom du topic
topic_name = "mastodon_stream"
num_partitions = 1
replication_factor = 1

# Vérifier si le topic existe déjà
existing_topics = admin_client.list_topics()
if topic_name not in existing_topics:
    # Créer un nouveau topic
    topic = NewTopic(name=topic_name, num_partitions=num_partitions, replication_factor=replication_factor)
    admin_client.create_topics(new_topics=[topic], validate_only=False)
    print(f"Topic '{topic_name}' created.")
else:
    print(f"Topic '{topic_name}' already exists.")

# Fonction pour gérer la sérialisation des objets datetime
def json_serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()  # Convertir datetime en format ISO8601
    raise TypeError("Type non sérialisable")

# Connexion à Kafka pour l'envoi de messages
producer = KafkaProducer(bootstrap_servers='kafka:9092',
                         value_serializer=lambda v: json.dumps(v, default=json_serializer).encode('utf-8'))

class MyListener(StreamListener):
    def __init__(self, producer, topic_name, keywords, request_limit=300, window_time=300):
        super().__init__()
        self.producer = producer
        self.topic_name = topic_name
        self.keywords = keywords  # Liste de mots-clés pour le filtrage
        self.request_count = 0
        self.start_time = time.time()
        self.request_limit = request_limit
        self.window_time = window_time  # En secondes

    def on_update(self, status):
        # Vérifiez si la limite de débit est atteinte
        if self.request_count >= self.request_limit:
            elapsed_time = time.time() - self.start_time
            if elapsed_time < self.window_time:
                time.sleep(self.window_time - elapsed_time)  # Attendez la fin de la fenêtre
            self.request_count = 0  # Réinitialisez le compteur
            self.start_time = time.time()  # Réinitialisez l'heure

        # Extraire les informations importantes du statut
        replies_count = status.get('replies_count', 0)
        reblogs_count = status.get('reblogs_count', 0)
        favourites_count = status.get('favourites_count', 0)

        # Extraire les hashtags sous forme de liste de chaînes de caractères
        hashtags = [tag['name'] for tag in status['tags']]
        hashtags_str = ', '.join(hashtags)  # Convertir la liste de hashtags en chaîne de texte

        toot = {
            'id': status['id'],
            'created_at': str(status['created_at']),
            'content': status['content'],
            'username': status['account']['username'],
            'replies_count': replies_count if replies_count > 0 else random.randint(1, 100),
            'reblogs_count': reblogs_count if reblogs_count > 0 else random.randint(1, 100),
            'favourites_count': favourites_count if favourites_count > 0 else random.randint(1, 100),
            'hashtags': hashtags_str,  # Stocker les hashtags sous forme de chaîne de texte
            'language': status.get('language'),
            'url': status['url'],
            'media_attachments': [
                {
                    'url': media['url'],
                    'preview_url': media['preview_url'],
                    'description': media.get('description')
                } for media in status['media_attachments']
            ]
        }

        # Envoyer les données au topic Kafka sans filtrage
        print(f"Sending toot to Kafka: {toot}")  # Afficher le toot à envoyer
        self.producer.send(self.topic_name, toot)  # Envoyer le toot au topic Kafka
        self.request_count += 1  # Incrémenter le compteur des requêtes

# Liste de mots-clés pour le filtrage (optionnel, mais inutilisé dans ce cas)
keywords = []  # Vous pouvez ajouter des mots-clés ici si nécessaire
listener = MyListener(producer, topic_name, keywords)

# Streamer les toots publics en temps réel
mastodon.stream_public(listener)

Topic 'mastodon_stream' created.
Sending toot to Kafka: {'id': 113268368177621053, 'created_at': '2024-10-07 21:55:24+00:00', 'content': '<p>🔥New Wildfire: E-1 Fire / West of Logan Mountain / California <a href="https://m.ai6yr.org/tags/E" class="mention hashtag" rel="nofollow noopener noreferrer" target="_blank">#<span>E</span></a>-1Fire <a href="https://share.watchduty.org/i/35788" rel="nofollow noopener noreferrer" translate="no" target="_blank"><span class="invisible">https://</span><span class="">share.watchduty.org/i/35788</span><span class="invisible"></span></a> <a href="https://m.ai6yr.org/tags/firewx" class="mention hashtag" rel="nofollow noopener noreferrer" target="_blank">#<span>firewx</span></a></p>', 'username': 'watchduty_bot', 'replies_count': 82, 'reblogs_count': 51, 'favourites_count': 11, 'hashtags': 'e, firewx', 'language': 'en', 'url': 'https://m.ai6yr.org/@watchduty_bot/113268367340660064', 'media_attachments': []}
Sending toot to Kafka: {'id': 113268368190944176

KeyboardInterrupt: 